# Alignment 2: Smith-Waterman Algorithm

The Needleman-Wunsch algorithm finds the optimal **global** alignment of the two given sequence.

At many times, however, a local alignment is more desirable than a global one. Say, you are mapping a short sequence (no more than 100 letters) onto a large sequence (say 1000 letters). If there is an exact match of the short sequence in the longer sequence, it is better to find this exact match. However the global Needle-Wunsch algorithm is easily influenced by sub-sequences out of the exact matching region, so that a different alignment is often produced.

~~~
sequence1 is 
sequence2 is
local alignment:
nw (global) alignment: check the previous notebook
~~~

The Smith-Waterman algorithm finds the optimal **local** alignment of the two given sequence. It can be seen as a variation of the Needleman-Wunsch algorithm. Two main distinctions are made:

- If the value of a cell in the score matrix is negative, it is set to zero;
- The trace back begins from the cell that has the highest score, and ends when a cell whose score is 0 is encountered.

## Implementation

In [ ]:
import numpy as np

In [ ]:
def smith_waterman(seq1:str, seq2:str, match:int=1, mismatch:int=0, gap:int=-1) -> None:
    
    '''
    Needleman-Wunsch aligment algorithm implementation, prints the aligment
    Args:
        seq1 (str): first sequence, it will appear on the first line of the aligment
        seq2 (str): second sequence
        match (int): score for a match
        mismatch (int): score for a mismatch
        gap (int): score for a gap, usually negative
    '''
    
    pass

## Tests

In [ ]:
seq1 = ''
seq2 = ''
smith_waterman(seq1, seq2)